In [2]:
import numpy as np

In [3]:
import sys
import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
Su = 0.061
R = ((8/3)**0.5)*Su #~.09

kappa_matrix = np.zeros((11,1,11))
kappa_matrix[5,0,5]=10e6

def case_four():
    nPoints = 200
    nCycles = 3
    omega   = 0.15
    case_four.time   = np.linspace(0, nCycles*2*np.pi/omega, nCycles*nPoints+1)
    case_four.stress = np.array([[0.000],
                       [0.000],
                       [0.000],
                       [0.050],
                       [0.000],
                       [0.000]])*np.sin( omega*case_four.time ) + 0.0*np.array([1,1,1,0,0,0]).reshape(6,1)*np.ones( case_four.time.shape )            
    case_four.StressDriven = 1          
    return case_four



case_four()
time, stress = case_four.time, case_four.stress

dev_stress = np.zeros(stress.shape)
for i in range(0,len(time)):
    dev_stress[:,i] = dev(stress[:,i])
#print(dev_stress)


CurStress = dev(np.array([0,0,0,.04,0,0])).reshape(6,1)
Stress0 = dev(np.array([0,0,0,0,0,0])).reshape(6,1)
zeta = CurStress - Stress0

#print(normS(CurStress))


#s_hat=s+k(s-s_o)= max deviatoric stress
#||s_hat||=||s + k(s-s_o)||=R

res_x = 101
res_y = res_x
center = np.array([np.floor(res_x/2), np.floor(res_y/2)])

maxIter = 1000

CurStress = CurStress.reshape(6,1)
Stress0 = Stress0.reshape(6,1)
zeta = CurStress - Stress0
norm_zeta = normS(zeta)

#Initialize
stress_matrix = np.zeros(kappa_matrix.shape)
#Must assign kappa_matrix cells to relative stress
for x in range(0,len(kappa_matrix)):
    for y in range(0,len(kappa_matrix)):
        #Initial assumption
        kappa = 0
        for i in range(0,maxIter):    
            norm_s_hat = normS(CurStress + kappa*(CurStress-Stress0))
            if R <= norm_s_hat:
                break
            elif R > norm_s_hat:
                kappa = kappa + .01
                kappa_matrix[x,0,y] = kappa
                
def concentric_assigner():
    #Initialize
    conc = np.zeros((res_x,res_y))

    for r in range(0, int(res_x/2)):   
        #While current ring radius is less than max
        #Output rings of 1s in matrix of 0s
        donut = ring(res_x,res_y,r) 
        for x in range(0,res_x):
            for y in range(0,res_y):
                if donut[y,x] == 1:
                    conc[y,x] = (np.floor(res_x/2)-r)*donut[y,x] 
                    #Fills in some interior empty cells as the program radiates outwards
                    if x < int(res_x/2) and conc[y,x+1] == 0:
                        conc[y,x+1] = (np.floor(res_x/2)-r)*donut[y,x] 
                    if x > int(res_x/2) and conc[y,x-1] == 0:
                        conc[y,x-1] = (np.floor(res_x/2)-r)*donut[y,x]                         
    return conc

#fig = plt.figure('Kappa Matrix')
#plt.plot(kappa_matrix[:,0,:],kappa_matrix[:,0,:])

#plt.imshow(kappa_matrix[:,0,:], cmap='hot', interpolation='nearest')
#plt.show()



def ring(res_x, res_y, radius):
    # xx and yy are (res_x, res_y) tables containing the x and y coordinates as values
    # mgrid is a mesh creation helper
    xx, yy = numpy.mgrid[:res_x, :res_y]
    # Circle contains the squared distance to the (res_x/2, res_y/2) point
    # Higher thresh allows for 'thicker' and more 'continous' circles but can overwrite previous values 
    #thresh = radius**1
    if radius < int(res_x/4):
        thresh = .95*radius
    else:
        thresh = 1.1*radius
    center = np.array([(np.floor(res_x/2)), (np.floor(res_y/2))])
    circle = (xx - center[0]) ** 2 + (yy - center[1]) ** 2
    # donut contains Trues and Falses organized in a ring shape
    donut = np.logical_and(circle < (radius**2 + thresh), circle > (radius**2 - thresh))
    donut = donut.astype(int)

    return donut

conc = concentric_assigner()
#print(conc)

im = plt.imshow(conc,cmap='inferno',interpolation='nearest')
plt.colorbar(im)
plt.show()

NameError: name 'dev' is not defined

In [ ]:
def polar2z(r,theta):
    return r * np.exp( 1j * theta )

def z2polar(z):
    return ( np.abs(z), np.angle(z) )



r, theta = z2polar(conc)
print(r)
pol = plt.imshow(theta,cmap='inferno',interpolation='nearest')
z = polar2z( r, theta )
x = np.real(z)
y = np.imag(z)
print('X: ',x.shape)
print('Y: ',y.shape)

In [5]:
#============================================================================
import numpy as np
from scipy.interpolate import interp1d
from scipy.ndimage import map_coordinates

import matplotlib
%matplotlib inline
matplotlib.use('Agg')
import matplotlib.pyplot as mpl

#============================================================================
def polar2cartesian(r, t, grid, x, y, order=3):
    X, Y = np.meshgrid(x, y)

    new_r = np.sqrt(X*X+Y*Y)
    new_t = np.arctan2(X, Y)

    ir = interp1d(r, np.arange(len(r)), bounds_error=False)
    it = interp1d(t, np.arange(len(t)))

    new_ir = ir(new_r.ravel())
    new_it = it(new_t.ravel())
    new_ir[new_r.ravel() > r.max()] = len(r)-1
    new_ir[new_r.ravel() < r.min()] = 0

    return map_coordinates(grid, np.array([new_ir, new_it]), order=order).reshape(new_r.shape)
#============================================================================
def cartesian2polar(x,y):
    X, Y = np.meshgrid(x,y)
    rho = np.sqrt((x)**2 + (y)**2)
    phi = np.arctan2(y,x)
    pass
#============================================================================

# Define original polar grid
nr = 33 #Radius', include center point as a radius
nt = 360 #Theta's

r = np.linspace(1, 33, nr)
t = np.linspace(-np.pi, np.pi, nt)
z_conc = np.flip(np.arange(nr*nt).reshape(nr,nt))
z_rand = np.random.random((nr,nt))
#print(z_conc)


# Define new cartesian grid
res_x = 65
res_y = res_x
x = np.linspace(-int(res_x/2), int(res_x/2), res_x)
y = np.linspace(-int(res_x/2), int(res_x/2), res_y)


#========================== DRAW FIGURES ==============================

# Interpolate polar grid to cartesian grid (nearest neighbor)
fig = mpl.figure()
ax = fig.add_subplot(111)
near = ax.imshow(polar2cartesian(r, t, z_conc, x, y, order=0), cmap = 'inferno',interpolation='nearest')
plt.colorbar(near)
#fig.savefig('test1.png')
#print(polar2cartesian(r,t,z_conc,x,y,order=3))

# Interpolate polar grid to cartesian grid (cubic spline)
fig = mpl.figure()
ax = fig.add_subplot(111)
cubic = ax.imshow(polar2cartesian(r, t, z_conc, x, y, order=3), cmap = 'inferno',interpolation='nearest')
plt.colorbar(cubic)

#============================================================================